# Introduction

Welcome to the **Tree Analysis** tutorial! 🌳

This tutorial will teach you how to use **regression trees** to understand how different factors (like age, gender, education) explain differences in sequence patterns. Think of it as building a decision tree that helps you understand: "What makes sequences different from each other?"

## What You'll Learn

By the end of this tutorial, you'll know how to:

1. **Compute basic statistics** - Calculate variance and test associations
2. **Build trees** - Create regression trees from distance matrices or sequence data
3. **Extract insights** - Get classification rules and assign sequences to groups
4. **Visualize results** - Plot and print trees to understand patterns

## Dataset: LSOG Dyadic Children

We'll use the **dyadic_children** dataset from the LSOG (Life Stories of Generations) study. This dataset contains sequences showing how children's living arrangements change from age 15 to 39. States represent different living situations (e.g., living with parents, alone, with partner).

## Prerequisites

- Basic Python knowledge
- Understanding of pandas DataFrames
- Familiarity with sequence data (if not, check out our [Quickstart Guide](../01_quickstart.ipynb))

Let's get started! 🚀

# Setup and Data Loading

First, let's import the necessary libraries and load our data.


In [ ]:
#| label: imports
#| echo: true
#| output: true
#| eval: true

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import Sequenzo modules
from sequenzo import SequenceData
from sequenzo.datasets import load_dataset
from sequenzo.dissimilarity_measures import get_distance_matrix

# Import all tree analysis functions
from sequenzo.tree_analysis import (
    # Basic utility functions
    compute_pseudo_variance,
    compute_distance_association,
    # Tree building functions
    build_distance_tree,
    build_sequence_tree,
    # Helper functions
    get_leaf_membership,
    get_classification_rules,
    assign_to_leaves,
    # Visualization functions
    plot_tree,
    print_tree,
    export_tree_to_dot,
)

print("✅ All libraries imported successfully!")

In [ ]:
#| label: load-data
#| echo: true
#| output: true
#| eval: true

# Load the dyadic_children dataset
df = load_dataset("dyadic_children")

# For this tutorial, we'll use the first 60 rows for faster computation
# In real analysis, you'd use the full dataset
df = df.head(60)

# Extract time columns (ages 15-39)
time_list = [c for c in df.columns if str(c).isdigit()]
time_list = sorted(time_list, key=int)

print(f"Dataset shape: {df.shape}")
print(f"Time columns: {len(time_list)} (ages {time_list[0]} to {time_list[-1]})")
print(f"\nFirst few rows:")
df.head()

In [ ]:
#| label: create-seqdata
#| echo: true
#| output: true
#| eval: true

# Create SequenceData object
# States are coded as 1-6 (different living arrangements)
seqdata = SequenceData(
    df,
    time=time_list,
    id_col="dyadID",
    states=[1, 2, 3, 4, 5, 6]
)

print(f"✅ SequenceData created: {seqdata.seqdata.shape[0]} sequences, {len(time_list)} time points")

In [ ]:
#| label: create-predictors
#| echo: true
#| output: true
#| eval: true

# Create predictor variables (covariates)
# These are the factors we want to test: do they explain differences in sequences?
np.random.seed(42)  # For reproducibility

predictors = pd.DataFrame({
    'gender': np.random.choice(['M', 'F'], size=len(df)),
    'age_group': np.random.choice(['young', 'middle', 'old'], size=len(df)),
    'education': np.random.choice(['low', 'medium', 'high'], size=len(df)),
    'income': np.random.randn(len(df)) * 10000 + 50000,  # Continuous variable
})

print("✅ Predictors created:")
print(predictors.head())
print(f"\nPredictor types:")
print(predictors.dtypes)

# Basic Utility Functions

Before building trees, let's learn about two fundamental functions that help us understand sequence variability and associations.

## Compute Pseudo-Variance (`compute_pseudo_variance`)

**What it does**: Measures how "spread out" sequences are in terms of their distances from each other. Think of it like variance, but for sequences!

**When to use**: When you want to know how diverse or similar your sequences are overall.

**Corresponds to TraMineR**: `dissvar()`


In [ ]:
#| label: compute-distance-matrix
#| echo: true
#| output: true
#| eval: true

# First, compute a distance matrix
# This tells us how "different" each pair of sequences is
print("[>] Computing distance matrix...")
dist_matrix = get_distance_matrix(
    seqdata=seqdata,
    method="LCS",  # Longest Common Subsequence
    norm="auto"
)

# Convert to numpy array if it's a DataFrame
if isinstance(dist_matrix, pd.DataFrame):
    dist_matrix = dist_matrix.values

print(f"✅ Distance matrix computed: {dist_matrix.shape[0]} x {dist_matrix.shape[1]}")

In [ ]:
#| label: pseudo-variance-basic
#| echo: true
#| output: true
#| eval: true

# Compute pseudo-variance (unweighted)
variance = compute_pseudo_variance(
    distance_matrix=dist_matrix,
    weights=None,  # Equal weights for all sequences
    squared=False  # Use raw distances, not squared
)

print(f"Pseudo-variance (unweighted): {variance:.6f}")
print(f"\nInterpretation: This value represents the average 'spread' of sequences.")
print(f"Higher values = more diverse sequences")
print(f"Lower values = more similar sequences")

In [ ]:
#| label: pseudo-variance-weighted
#| echo: true
#| output: true
#| eval: true

# Compute pseudo-variance with weights
# This is useful when some sequences are more important than others
weights = np.ones(len(df)) * 2.0  # All sequences have weight 2

variance_weighted = compute_pseudo_variance(
    distance_matrix=dist_matrix,
    weights=weights,
    squared=False
)

print(f"Pseudo-variance (weighted, all weights=2): {variance_weighted:.6f}")
print(f"\nNote: With equal weights, weighted variance ≈ unweighted variance")

In [ ]:
#| label: pseudo-variance-squared
#| echo: true
#| output: true
#| eval: true

# Compute pseudo-variance with squared distances
# Squaring emphasizes larger differences
variance_squared = compute_pseudo_variance(
    distance_matrix=dist_matrix,
    weights=None,
    squared=True  # Square distances before computing variance
)

print(f"Pseudo-variance (squared): {variance_squared:.6f}")
print(f"Pseudo-variance (normal): {variance:.6f}")
print(f"\nNote: Squared variance is typically larger because it emphasizes big differences")

## Compute Distance Association (`compute_distance_association`)

**What it does**: Tests whether a grouping variable (like gender or age group) explains differences in sequence patterns. It's like ANOVA, but for sequences!

**When to use**: When you want to know if sequences in different groups (e.g., men vs. women) are significantly different.

**Corresponds to TraMineR**: `dissassoc()`


In [ ]:
#| label: association-basic
#| echo: true
#| output: true
#| eval: true

# Test association between sequences and gender
# This tells us: do men and women have different sequence patterns?
groups = predictors['gender'].values

result = compute_distance_association(
    distance_matrix=dist_matrix,
    group=groups,
    weights=None,
    R=100,  # Number of permutations for significance test (smaller for speed)
    weight_permutation="none",  # No special weight handling
    squared=False
)

print("=== Association Test Results ===")
print(f"Pseudo F-statistic: {result['pseudo_f']:.4f}")
print(f"Pseudo R²: {result['pseudo_r2']:.4f}")
print(f"P-value: {result['pseudo_f_pval']:.4f}")
print(f"\nInterpretation:")
print(f"- Pseudo R²: {result['pseudo_r2']*100:.1f}% of variance explained by gender")
print(f"- P-value: {'Significant' if result['pseudo_f_pval'] < 0.05 else 'Not significant'} (p < 0.05)")

In [ ]:
#| label: association-details
#| echo: true
#| output: true
#| eval: true

# Look at detailed results
print("\n=== Group Statistics ===")
print(result['groups'])

print("\n=== ANOVA Table ===")
print(result['anova_table'])

# Building Regression Trees

Now for the exciting part! We'll build regression trees that automatically find which factors best explain differences in sequences.

## Build Distance Tree (`build_distance_tree`)

**What it does**: Builds a tree from a pre-computed distance matrix. Use this when you already have distances calculated.

**When to use**: When you want to reuse a distance matrix or have computed distances using a specific method.

**Corresponds to TraMineR**: `disstree()`


In [ ]:
#| label: build-distance-tree
#| echo: true
#| output: true
#| eval: true

# Build a distance tree
# The tree will automatically find the best splits based on predictors
tree = build_distance_tree(
    distance_matrix=dist_matrix,
    predictors=predictors,
    weights=None,
    min_size=0.1,  # Minimum 10% of sequences in each node
    max_depth=3,   # Maximum 3 levels deep
    R=100,         # Permutations for significance testing
    pval=0.1,     # Only keep splits with p-value < 0.1
    weight_permutation="none",
    squared=False
)

print("✅ Tree built successfully!")
print(f"\nTree structure:")
print(f"- Total sequences: {tree['info']['n']}")
print(f"- Number of leaves: {tree['fitted']['(fitted)'].nunique()}")
print(f"- Global R²: {tree['info'].get('R2', 0):.4f}")

In [ ]:
#| label: tree-fitted
#| echo: true
#| output: true
#| eval: true

# Look at which sequences belong to which leaf nodes
print("\n=== Sequence Assignments ===")
print("Each sequence is assigned to a leaf node:")
print(tree['fitted'].head(10))

## Build Sequence Tree (`build_sequence_tree`)

**What it does**: Builds a tree directly from sequence data. It automatically computes distances for you!

**When to use**: When you want a simpler workflow - just pass sequences and predictors.

**Corresponds to TraMineR**: `seqtree()`


In [ ]:
#| label: build-sequence-tree
#| echo: true
#| output: true
#| eval: true

# Build a sequence tree
# This is easier - we don't need to compute distances first!
seqtree = build_sequence_tree(
    seqdata=seqdata,
    predictors=predictors,
    distance_matrix=None,  # Will be computed automatically
    distance_method="LCS",
    distance_params={'norm': 'auto'},
    weighted=True,  # Use weights from seqdata if available
    min_size=0.1,
    max_depth=3,
    R=100,
    pval=0.1,
    weight_permutation="replicate",
    squared=False
)

print("✅ Sequence tree built successfully!")
print(f"\nTree structure:")
print(f"- Total sequences: {seqtree['info']['n']}")
print(f"- Number of leaves: {seqtree['fitted']['(fitted)'].nunique()}")
print(f"- Global R²: {seqtree['info'].get('R2', 0):.4f}")

# Extracting Information from Trees

Once you've built a tree, you'll want to extract useful information from it!

## Get Leaf Membership (`get_leaf_membership`)

**What it does**: Gets which leaf node each sequence belongs to. You can get either numeric IDs or human-readable labels.

**When to use**: When you want to know which group each sequence belongs to.

**Corresponds to TraMineR**: `disstreeleaf()`


In [ ]:
#| label: leaf-membership-ids
#| echo: true
#| output: true
#| eval: true

# Get leaf IDs (numeric)
leaf_ids = get_leaf_membership(tree, label=False)

print("Leaf IDs (first 10 sequences):")
print(leaf_ids[:10])
print(f"\nUnique leaves: {len(np.unique(leaf_ids))}")
print(f"\nDistribution:")
print(pd.Series(leaf_ids).value_counts().sort_index())

In [ ]:
#| label: leaf-membership-labels
#| echo: true
#| output: true
#| eval: true

# Get leaf labels (human-readable)
leaf_labels = get_leaf_membership(tree, label=True)

print("Leaf labels (first 10 sequences):")
print(leaf_labels[:10])
print(f"\nUnique labels:")
print(leaf_labels.unique())

## Get Classification Rules (`get_classification_rules`)

**What it does**: Converts the tree structure into Python-readable rules. Each rule tells you what conditions lead to each leaf node.

**When to use**: When you want to understand or explain the tree structure, or assign new sequences to leaves.

**Corresponds to TraMineR**: `disstree.get.rules()`


In [ ]:
#| label: classification-rules
#| echo: true
#| output: true
#| eval: true

# Get classification rules
rules = get_classification_rules(tree)

print(f"Number of rules: {len(rules)}")
print(f"\nClassification Rules:")
for i, rule in enumerate(rules, 1):
    print(f"\nRule {i}:")
    print(f"  {rule}")
    print(f"  Interpretation: Sequences matching this rule belong to leaf {i}")

## Assign to Leaves (`assign_to_leaves`)

**What it does**: Uses classification rules to assign new sequences (or existing ones) to leaf nodes based on their predictor values.

**When to use**: When you want to classify new sequences or verify assignments.

**Corresponds to TraMineR**: `disstree.assign()`


In [ ]:
#| label: assign-to-leaves
#| echo: true
#| output: true
#| eval: true

# Assign sequences to leaves using rules
# This uses the same predictors we used to build the tree
assignments = assign_to_leaves(rules, predictors)

print("Assignments (first 10 sequences):")
print(assignments[:10])
print(f"\nNote: Each number corresponds to a rule/leaf node")
print(f"NaN means the sequence doesn't match any rule")

In [ ]:
#| label: verify-assignments
#| echo: true
#| output: true
#| eval: true

# Verify that assignments match the tree's fitted values
tree_assignments = tree['fitted']['(fitted)'].values

print("Comparing assignments:")
print(f"From tree: {tree_assignments[:5]}")
print(f"From rules: {assignments[:5]}")
print(f"\nMatch: {np.array_equal(tree_assignments, assignments)}")

# Visualizing Trees

Visualization is crucial for understanding tree structures! Sequenzo provides three ways to visualize trees.

## Print Tree (`print_tree`)

**What it does**: Prints a text-based representation of the tree in your console/notebook.

**When to use**: Quick visualization, or when you want a simple text output.

**Corresponds to TraMineR**: `print.disstree()` and `print.seqtree()`


In [ ]:
#| label: print-tree
#| echo: true
#| output: true
#| eval: true

# Print tree structure
print_tree(tree, digits=3, medoid=False)

## Plot Tree (`plot_tree`)

**What it does**: Creates a graphical visualization of the tree using matplotlib.

**When to use**: When you want a visual representation for presentations or papers.

**Corresponds to TraMineR**: `plot.disstree()` and `plot.seqtree()`


In [ ]:
#| label: plot-tree
#| echo: true
#| output: true
#| eval: true

# Plot tree
plot_tree(
    tree,
    filename=None,  # Don't save to file
    show_tree=True,  # Display the plot
    show_depth=False,
    figsize=(12, 8),
    dpi=100
)

## Export to DOT (`export_tree_to_dot`)

**What it does**: Exports the tree to GraphViz DOT format, which can be rendered as high-quality images.

**When to use**: When you need publication-quality figures or want to customize the visualization.

**Note**: Requires `graphviz` package to be installed.

**Corresponds to TraMineR**: `disstree2dot()` and `seqtree2dot()`


In [ ]:
#| label: export-dot
#| echo: true
#| output: true
#| eval: true

# Export to DOT format
# Note: This requires graphviz package
try:
    export_tree_to_dot(
        tree,
        filename="tree_analysis_lsog",
        show_depth=False
    )
    print("\n✅ DOT file created!")
    print("To render: dot -Tpng tree_analysis_lsog.dot -o tree_analysis_lsog.png")
except ImportError:
    print("⚠️  graphviz not installed. Skipping DOT export.")
    print("Install with: pip install graphviz")

# Complete Workflow Example

Let's put it all together! Here's a complete example showing the typical workflow:


In [ ]:
#| label: complete-workflow
#| echo: true
#| output: true
#| eval: true

# Step 1: Load and prepare data
df = load_dataset("dyadic_children").head(60)
time_list = sorted([c for c in df.columns if str(c).isdigit()], key=int)
seqdata = SequenceData(df, time=time_list, id_col="dyadID", states=[1, 2, 3, 4, 5, 6])

# Step 2: Create predictors
np.random.seed(42)
predictors = pd.DataFrame({
    'gender': np.random.choice(['M', 'F'], size=len(df)),
    'education': np.random.choice(['low', 'medium', 'high'], size=len(df)),
})

# Step 3: Build tree
tree = build_sequence_tree(
    seqdata=seqdata,
    predictors=predictors,
    min_size=0.1,
    max_depth=3,
    R=100,
    pval=0.1
)

# Step 4: Extract information
leaf_ids = get_leaf_membership(tree, label=False)
rules = get_classification_rules(tree)

# Step 5: Visualize
print_tree(tree)

print(f"\n✅ Complete workflow finished!")
print(f"Tree has {len(np.unique(leaf_ids))} leaf nodes")
print(f"Found {len(rules)} classification rules")

# Summary and Next Steps

Congratulations! 🎉 You've learned how to:

1. ✅ Compute pseudo-variance to measure sequence diversity
2. ✅ Test associations between groups and sequence patterns
3. ✅ Build regression trees from distance matrices and sequence data
4. ✅ Extract leaf memberships and classification rules
5. ✅ Visualize trees using text, plots, and DOT export

## Key Takeaways

- **Pseudo-variance** tells you how diverse your sequences are
- **Association tests** tell you if groups differ significantly
- **Trees** automatically find which factors explain sequence differences
- **Rules** help you understand and explain tree structures
- **Visualization** makes trees easy to understand and present

## What's Next?

- Try different distance measures (OM, HAM, DHD)
- Experiment with different predictor variables
- Adjust tree parameters (min_size, max_depth, pval)
- Combine tree analysis with clustering (see [Cluster Analysis Tutorial](../cluster_analysis/))
- Explore other Sequenzo modules!

## Further Reading

- TraMineR documentation: https://traminer.unige.ch/
- Studer et al. (2011): Discrepancy analysis of state sequences

Happy analyzing! 🌳✨